<a href="https://colab.research.google.com/github/maahieummah/30-Days-Map-Challenge/blob/main/30DayMapChallenge_(3)polygons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import geopandas as gpd
import folium
from folium.features import GeoJsonTooltip
import matplotlib.cm as cm
import matplotlib.colors as colors

# ======================================================
# 1️⃣ Load J&K District Boundary Data (GADM)
# ======================================================
url = "https://geodata.ucdavis.edu/gadm/gadm4.1/shp/gadm41_IND_shp.zip"

print("📍 Loading district boundaries from GADM...")
india = gpd.read_file(url, layer="gadm41_IND_2")

# Filter Jammu & Kashmir (Union Territory)
jk = india[india["NAME_1"] == "Jammu and Kashmir"].to_crs(epsg=4326)

# ======================================================
# 2️⃣ Compute Centroids for Label Placement
# ======================================================
jk["centroid"] = jk.geometry.centroid

# ======================================================
# 3️⃣ Assign Unique Colors per District
# ======================================================
cmap = cm.get_cmap("tab20", len(jk))
color_list = [colors.to_hex(cmap(i)) for i in range(len(jk))]
jk["color"] = color_list

# ======================================================
# 4️⃣ Initialize Interactive Map with Base Layers
# ======================================================
m = folium.Map(location=[33.5, 75.0], zoom_start=7, tiles=None)

# Add basemap options with attribution where required
folium.TileLayer("CartoDB positron", name="Light Map").add_to(m)
folium.TileLayer("OpenStreetMap", name="OpenStreetMap").add_to(m)
# Add attribution for Stamen tiles
stamen_attribution = "Map tiles by Stamen Design, under CC BY 3.0. Data by OpenStreetMap, under ODbL."
folium.TileLayer("Stamen Terrain", name="Terrain", attr=stamen_attribution).add_to(m)
folium.TileLayer("Stamen Toner", name="Black & White", attr=stamen_attribution).add_to(m)

# ======================================================
# 5️⃣ Add District Boundaries with Unique Colors
# ======================================================
geojson_data = jk.drop(columns=["centroid"]).to_json()

def style_function(feature):
    district_name = feature["properties"]["NAME_2"]
    color = jk.loc[jk["NAME_2"] == district_name, "color"].values[0]
    return {
        "fillColor": color,
        "color": "black",
        "weight": 1,
        "fillOpacity": 0.6,
    }

# Add interactive popups + tooltip
folium.GeoJson(
    geojson_data,
    name="Jammu & Kashmir Districts",
    style_function=style_function,
    highlight_function=lambda x: {"weight": 3, "color": "yellow"},
    tooltip=GeoJsonTooltip(
        fields=["NAME_2"],
        aliases=["District:"],
        localize=True,
        sticky=True,
        labels=True,
    ),
    popup=folium.GeoJsonPopup(
        fields=["NAME_2"],
        aliases=["District:"],
        localize=True
    )
).add_to(m)

# ======================================================
# 6️⃣ Add District Labels (Permanent Text)
# ======================================================
for _, row in jk.iterrows():
    lon, lat = row["centroid"].x, row["centroid"].y
    folium.map.Marker(
        [lat, lon],
        icon=folium.DivIcon(
            html=f"""
            <div style="font-size: 9pt;
                        color: black;
                        font-weight: bold;
                        text-shadow: 1px 1px 1px white;
                        white-space: nowrap;">
                {row['NAME_2']}
            </div>
            """
        )
    ).add_to(m)

# ======================================================
# 7️⃣ Add Layer Control & Mini Legend
# ======================================================
folium.LayerControl(position="topright").add_to(m)

legend_html = """
<div style="
     position: fixed;
     bottom: 20px; left: 20px; width: 250px; height: 60px;
     background-color: white;
     border:2px solid grey;
     z-index:9999;
     font-size:12px;
     padding: 5px;">
<b>🧭 Map Info:</b><br>
Click a district to view its name.<br>
Use top-right control to switch map layers.
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

# ======================================================
# 8️⃣ Save and Display Map
# ======================================================
m.save("jk_districts_ui_panel.html")
print("✅ Saved: jk_districts_ui_panel.html")


📍 Loading district boundaries from GADM...


/tmp/ipython-input-2731249816.py:21: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  jk["centroid"] = jk.geometry.centroid
/tmp/ipython-input-2731249816.py:26: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap("tab20", len(jk))


✅ Saved: jk_districts_ui_panel.html
